In [1]:
from load_results import load_result_dataset
import pandas as pd
import numpy as np
pn1 = 'full_fine_tuning_50epochs_edge_paper_final2'
pn2 = 'full_fine_tuning_50epochs_paper_final2'

final_data = load_result_dataset(pn1, pn2)
final_data = [{**d, 'ft_strategy': 'FFT (50 epochs)'} for d in final_data]

../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion_aesthetic-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion2B-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/deit_small_patch16_224.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/robust_resnet50_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/vit_small_patch16_224.augreg_in21k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/convnext_base.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/resnet50.a1_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/robust_vit_base_patch16_224_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_

In [2]:
df = pd.DataFrame(final_data)
for arch_size in ('small', 'medium', 'large'):
    df_curr = df[ df.model_size == arch_size ]
    nan_percentage = (df_curr.isna().sum().sum() / df_curr.size) * 100
    print("Percentage of NaN values, for {} architecture: {}".format(arch_size, nan_percentage) )

Percentage of NaN values, for small architecture: 1.1904761904761905
Percentage of NaN values, for medium architecture: 0.0
Percentage of NaN values, for large architecture: 0.0


In [13]:
import pandas as pd
import yaml

df = pd.DataFrame(final_data)

def get_restart_job(row):

    if row.isna().any():

        missing = {col: row[col] for col in ['clean_acc', 'Linf_acc', 'L1_acc', 'L2_acc', 'common_acc']}
        print(missing, missing.values(), all( list(missing.values()) ))

        if all(missing.values()):
            return "job1_hpo.sh"
        elif not missing['clean_acc'] and not missing['Linf_acc'] and missing['L1_acc']:
            return "job4_test_l1.sh"
        elif not missing['clean_acc'] and not missing['Linf_acc'] and not missing['L1_acc'] and missing['L2_acc']:
            return "job5_test_l2.sh"
        elif not missing['clean_acc'] and not missing['Linf_acc'] and not missing['L1_acc'] and not missing['L2_acc'] and missing['common_acc']:
            return "job6_test_common.sh"
    else:
        return None  # means no job needs to be restarted

df['restart_from'] = df.apply(get_restart_job, axis=1)
to_restart = df[df['restart_from'].notna()][['backbone', 'dataset', 'loss_function', 'restart_from', 'model_size']]
to_restart
to_restart.to_csv("./restart_max.csv")

{'clean_acc': nan, 'Linf_acc': nan, 'L1_acc': 0.34, 'L2_acc': 0.454, 'common_acc': 0.48} dict_values([nan, nan, 0.34, 0.454, 0.48]) True
{'clean_acc': 0.912, 'Linf_acc': 0.763, 'L1_acc': nan, 'L2_acc': nan, 'common_acc': nan} dict_values([0.912, 0.763, nan, nan, nan]) True
{'clean_acc': nan, 'Linf_acc': nan, 'L1_acc': 0.038, 'L2_acc': 0.08, 'common_acc': 0.159} dict_values([nan, nan, 0.038, 0.08, 0.159]) True
{'clean_acc': nan, 'Linf_acc': nan, 'L1_acc': 0.764, 'L2_acc': 0.811, 'common_acc': 0.801} dict_values([nan, nan, 0.764, 0.811, 0.801]) True


In [5]:
from process_database import process_grouped_df, process_rankings, compute_odds_ratio_by_group

size = "small"
grouped_df = process_grouped_df(final_data, size)
grouped_df = process_rankings(grouped_df)
grouped_df
# grouped_df.sort_values(by="sum", ascending=False)

Percentage of NaN values: 0.21%


dataset                                                                                                                                    TOTAL  \
metric                                                                                                                                     borda   
backbone                      backbone_name            loss_function pre_training_strategy model_type          model_size ft_strategy              
edgenext_small.usi_in1k       edgenetx_s,sup,in1k      TRADES_v2     supervised            hybrid              small      FFT (50 epochs)  372.0   
coat_tiny.in1k                coat_t,sup,in1k          TRADES_v2     supervised            hybrid              small      FFT (50 epochs)  325.0   
deit_tiny_patch16_224.fb_in1k deit_t,sup,in1k          TRADES_v2     supervised            fully attention     small      FFT (50 epochs)  256.0   
regnetx_004.pycls_in1k        regnetx_004,sup,in1k     CLASSIC_AT    supervised            fully convolutional small      FFT (50 epochs)  252.0   
                                                       TRADES_v2     supervised            fully convolutional small      FFT (50 epochs)  158.0   
mobilenetv3_large_100.ra_in1k mobilenet_v3,sup,in1k    TRADES_v2     supervised            fully convolutional small      FFT (50 epochs)  178.0   
edgenext_small.usi_in1k       edgenetx_s,sup,in1k      CLASSIC_AT    supervised            hybrid              small      FFT (50 epochs)  277.0   
mobilenetv3_large_100.ra_in1k mobilenet_v3,sup,in1k    CLASSIC_AT    supervised            fully convolutional small      FFT (50 epochs)  209.0   
deit_tiny_patch16_224.fb_in1k deit_t,sup,in1k          CLASSIC_AT    supervised            fully attention     small      FFT (50 epochs)  167.0   
coat_tiny.in1k                coat_t,sup,in1k          CLASSIC_AT    supervised            hybrid              small      FFT (50 epochs)  158.0   
mobilevit-small               mobilevit_s,sup,in1k     CLASSIC_AT    supervised            hybrid              small      FFT (50 epochs)  177.0   
efficientnet-b0               efficientnet_b0,sup,in1k CLASSIC_AT    supervised            fully convolutional small      FFT (50 epochs)  124.0   
                                                       TRADES_v2     supervised            fully convolutional small      FFT (50 epochs)   91.0   
mobilevit-small               mobilevit_s,sup,in1k     TRADES_v2     supervised            hybrid              small      FFT (50 epochs)  139.0   

dataset                                                                                                                                                 \
metric                                                                                                                                    nan_geom_cnt   
backbone                      backbone_name            loss_function pre_training_strategy model_type          model_size ft_strategy                    
edgenext_small.usi_in1k       edgenetx_s,sup,in1k      TRADES_v2     supervised            hybrid              small      FFT (50 epochs)            0   
coat_tiny.in1k                coat_t,sup,in1k          TRADES_v2     supervised            hybrid              small      FFT (50 epochs)            1   
deit_tiny_patch16_224.fb_in1k deit_t,sup,in1k          TRADES_v2     supervised            fully attention     small      FFT (50 epochs)            0   
regnetx_004.pycls_in1k        regnetx_004,sup,in1k     CLASSIC_AT    supervised            fully convolutional small      FFT (50 epochs)            0   
                                                       TRADES_v2     supervised            fully convolutional small      FFT (50 epochs)            0   
mobilenetv3_large_100.ra_in1k mobilenet_v3,sup,in1k    TRADES_v2     supervised            fully convolutional small      FFT (50 epochs)            0   
edgenext_small.usi_in1k       edgenetx_s,sup,in1k      CLASSIC_AT    supervised            hybrid

In [6]:
from process_database import process_grouped_df, process_rankings, compute_odds_ratio_by_group

grouped_df = process_grouped_df(final_data, size="small")
grouped_df = process_rankings(grouped_df)
grouped_df

Percentage of NaN values: 0.49%


dataset                                                                                                                                    TOTAL  \
metric                                                                                                                                     borda   
backbone                      backbone_name            loss_function pre_training_strategy model_type          model_size ft_strategy              
edgenext_small.usi_in1k       edgenetx_s,sup,in1k      TRADES_v2     supervised            hybrid              small      FFT (50 epochs)  370.0   
deit_tiny_patch16_224.fb_in1k deit_t,sup,in1k          TRADES_v2     supervised            fully attention     small      FFT (50 epochs)  256.0   
regnetx_004.pycls_in1k        regnetx_004,sup,in1k     CLASSIC_AT    supervised            fully convolutional small      FFT (50 epochs)  252.0   
coat_tiny.in1k                coat_t,sup,in1k          TRADES_v2     supervised            hybrid              small      FFT (50 epochs)  261.0   
regnetx_004.pycls_in1k        regnetx_004,sup,in1k     TRADES_v2     supervised            fully convolutional small      FFT (50 epochs)  158.0   
mobilenetv3_large_100.ra_in1k mobilenet_v3,sup,in1k    TRADES_v2     supervised            fully convolutional small      FFT (50 epochs)  177.0   
edgenext_small.usi_in1k       edgenetx_s,sup,in1k      CLASSIC_AT    supervised            hybrid              small      FFT (50 epochs)  276.0   
mobilenetv3_large_100.ra_in1k mobilenet_v3,sup,in1k    CLASSIC_AT    supervised            fully convolutional small      FFT (50 epochs)  208.0   
deit_tiny_patch16_224.fb_in1k deit_t,sup,in1k          CLASSIC_AT    supervised            fully attention     small      FFT (50 epochs)  167.0   
coat_tiny.in1k                coat_t,sup,in1k          CLASSIC_AT    supervised            hybrid              small      FFT (50 epochs)  158.0   
mobilevit-small               mobilevit_s,sup,in1k     CLASSIC_AT    supervised            hybrid              small      FFT (50 epochs)  177.0   
efficientnet-b0               efficientnet_b0,sup,in1k CLASSIC_AT    supervised            fully convolutional small      FFT (50 epochs)  124.0   
                                                       TRADES_v2     supervised            fully convolutional small      FFT (50 epochs)   91.0   
mobilevit-small               mobilevit_s,sup,in1k     TRADES_v2     supervised            hybrid              small      FFT (50 epochs)  139.0   

dataset                                                                                                                                                 \
metric                                                                                                                                    nan_geom_cnt   
backbone                      backbone_name            loss_function pre_training_strategy model_type          model_size ft_strategy                    
edgenext_small.usi_in1k       edgenetx_s,sup,in1k      TRADES_v2     supervised            hybrid              small      FFT (50 epochs)            0   
deit_tiny_patch16_224.fb_in1k deit_t,sup,in1k          TRADES_v2     supervised            fully attention     small      FFT (50 epochs)            0   
regnetx_004.pycls_in1k        regnetx_004,sup,in1k     CLASSIC_AT    supervised            fully convolutional small      FFT (50 epochs)            0   
coat_tiny.in1k                coat_t,sup,in1k          TRADES_v2     supervised            hybrid              small      FFT (50 epochs)            2   
regnetx_004.pycls_in1k        regnetx_004,sup,in1k     TRADES_v2     supervised            fully convolutional small      FFT (50 epochs)            0   
mobilenetv3_large_100.ra_in1k mobilenet_v3,sup,in1k    TRADES_v2     supervised            fully convolutional small      FFT (50 epochs)            0   
edgenext_small.usi_in1k       edgenetx_s,sup,in1k      CLASSIC_AT    supervised            hybrid

In [5]:

from process_database import process_grouped_df, process_rankings, compute_odds_ratio_by_group
import pandas as pd

odds_loss_list = []
odds_architecture_list = []
odds_pretraining_list = []

for size_id,size in [ (0,'small'),(1,'medium'),(2,'large')]:
    grouped_df = process_grouped_df(final_data, size)
    grouped_df = process_rankings(grouped_df)

    odds_df = grouped_df['TOTAL']
    odds_df = odds_df.reset_index()
    odds_df.to_csv("./odds_ratio.csv")

    # Define tier1 threshold based on the number of rows
    n_rows = odds_df.shape[0]
    top_k = int(np.ceil(0.33 * n_rows))  # top 10% as tier1

    # Apply tier1 flag
    odds_df["in_tier1"] = 0
    odds_df.loc[odds_df.sort_values(by="score_sum", ascending=False).head(top_k).index, "in_tier1"] = 1

    # Compute odds ratios for model_type and loss_function
    odds_model_type = compute_odds_ratio_by_group(odds_df, "model_type")
    odds_pretrain_type = compute_odds_ratio_by_group(odds_df, "pre_training_strategy")
    odds_loss_function = compute_odds_ratio_by_group(odds_df, "loss_function")
    odds_loss_function['size'] = size
    odds_pretrain_type['size'] = size
    odds_model_type['size'] = size

    odds_loss_list.append(odds_loss_function)
    odds_architecture_list.append(odds_model_type)
    odds_pretraining_list.append(odds_pretrain_type)

combined_odds_loss = pd.concat(odds_loss_list, axis=0)
combined_odds_architecture = pd.concat(odds_architecture_list, axis=0)
combined_odds_pretraining = pd.concat(odds_pretraining_list, axis=0)

combined_odds_pretraining = combined_odds_pretraining.fillna(0)
combined_odds_pretraining = combined_odds_pretraining[combined_odds_pretraining["size"].isin(["medium", "large"])]


Percentage of NaN values: 3.75%
Percentage of NaN values: 3.75%
Percentage of NaN values: 3.75%


In [6]:
import plotly.express as px

# Create mapping from numeric size to categorical label
size_map = {"small":0, "medium":1, "large":2}

# Convert numeric size columns to categorical labels
combined_odds_loss["size_label"] = combined_odds_loss["size"].map(size_map)
combined_odds_architecture["size_label"] = combined_odds_architecture["size"].map(size_map)
combined_odds_pretraining["size_label"] = combined_odds_pretraining["size"].map(size_map)

plots = [
    ("loss_function", combined_odds_loss),
    ("model_type", combined_odds_architecture),
    ("pre_training_strategy", combined_odds_pretraining)
]


figs = []

for var, plot_df in plots:

    plot_df = plot_df.copy()
    plot_df["odds_ratio"] = plot_df["odds_ratio"].replace(0, 0)

    # plot_df["text_label"] = plot_df.apply(
    #     lambda row: (
    #         f'{int(row["odds_ratio"])}' if row["odds_ratio"] == int(row["odds_ratio"])
    #         else f'{row["odds_ratio"]:.2f}'
    #     ) + f'  (p:{row["p_value"]:.3f})',
    #     axis=1
    # )
    plot_df["text_label"] = plot_df.apply(
        lambda row: f'{int(row["odds_ratio"])}' if row["odds_ratio"] == int(row["odds_ratio"])
        else f'{row["odds_ratio"]:.2f}', axis=1 )
    plot_df["text_label"] = plot_df.apply(
        lambda row: f'{row["text_label"]}<br>(p={row["p_value"]:.3f})', axis=1)
            
    fig = px.bar(
        plot_df,
        x="size",
        y="odds_ratio",
        color=var,
        barmode="group",
        text="text_label",
        labels={"size_label": "Model Size", "odds_ratio": "Odds Ratio"},
    )

    fig.update_layout(yaxis=dict(range=[0, 25]),)

    fig.update_traces(
        textposition="outside",
        textangle=-90,  # Rotate text vertically
        textfont=dict(
            color="black",
            family="Arial"
        ),
        cliponaxis=False
    )

    if var == 'loss_function':
        fig.update_layout(
        uniformtext_minsize=11,  # or your desired minimum font size
        uniformtext_mode='show',  # force showing all text
        )
    else:
        fig.update_layout(
        uniformtext_minsize=7,  # or your desired minimum font size
        uniformtext_mode='show',  # force showing all text
        )



    fig.update_layout(
            margin=dict(l=0, r=0, t=0, b=0),  # remove all outer margins
            width=450,    # in pixels (~2.5 inches at 96 DPI)
            height=400,    # in pixels (~1.875 inches)
            yaxis_title="Odds Ratio",
            xaxis_title="Model Size",
            legend_title_text=var,
            
            # White background
            plot_bgcolor="white",
            paper_bgcolor="white",
            
            # Light grey grid
            xaxis=dict(showgrid=False, gridcolor="lightgrey"),
            yaxis=dict(showgrid=True, gridcolor="lightgrey"),
            
            # Legend inside figure
            legend=dict(    
                font=dict(size=12),  # Adjust font size here 
                yanchor="top",
                xanchor="right",
                bgcolor="rgba(255,255,255,1)",  # semi-transparent white box
                bordercolor="lightgrey",
                borderwidth=1
            )
        )
    fig.show()
    fig.write_image("./paper_figures/oddsratio_{}_{}.png".format(pn2,var), scale=3  )
    
    

In [14]:

from matplotlib import cm
from matplotlib.colors import to_hex
import re


import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px


import pandas as pd

df1 = grouped_df.copy()

metrics_to_sum = ["L1_acc", "L2_acc", "Linf_acc", "clean_acc", "common_acc"]

for metric in metrics_to_sum:
    metric_cols = [col for col in df1.columns if col[1] == metric]
    df1[f'sum_{metric}'] = df1[metric_cols].sum(axis=1)

df_per_row_sums = df1[[f'sum_{m}' for m in metrics_to_sum]]

df_per_row_sums.columns = df_per_row_sums.columns.get_level_values(0)

df_per_row_sums.columns.name = 'metric'

df_per_row_sums


# Your model type mapping
model_type_map = {
    'convnext_base': "fully convolutional",
    'convnext_tiny': "fully convolutional",
    'deit_small': "fully attention",
    'vit_base': "fully attention",
    'vit_small': "fully attention",
    'resnet50': "fully convolutional",
    'eva02_base': "fully attention",
    'eva02_tiny': "fully attention",
    'swin_base': "fully attention",
    'swin_tiny': "fully attention",
    'coatnet_0': "hybrid",
    'coatnet_2': "hybrid",
    'regnetx_004': "fully convolutional",
    'efficientnet-b0': "fully convolutional", 
    'deit_tiny': "fully attention",
    'mobilevit-small': "hybrid",
    'mobilenetv3': "fully convolutional",
    'edgenext_small': "fully convolutional",
    'coat_tiny': "hybrid",
}

# Colormap for each model type
type_to_cmap = {
    'fully convolutional': cm.Purples,     # deep violet → lavender
    'fully attention': cm.Oranges,         # strong orange → light peach
    'hybrid': cm.Greens                    # forest → mint
}

# Extract base backbone name from full backbone string
def extract_base_name(backbone_name):
    for base in model_type_map:
        if base in backbone_name:
            return base
    return 'unknown'

# Get all unique backbones
backbones = sorted(set(backbone for backbone, _, _, _ in df_per_row_sums.index))

# Map backbones to model type
backbone_model_type = {b: model_type_map.get(extract_base_name(b), 'unknown') for b in backbones}

# Assign gradient color per backbone based on model type
color_map = {}
for model_type in ['fully convolutional', 'fully attention', 'hybrid']:
    bks = [bk for bk in backbones if backbone_model_type[bk] == model_type]
    n = len(bks)
    for i, bk in enumerate(bks):
        start, end = 0.5, 0.9  # avoid very light or very dark edges
        position = start + (end - start) * (i / max(n - 1, 1))
        color = to_hex(type_to_cmap[model_type](position))
        color_map[bk] = color


# Define metrics
metrics = ['sum_L1_acc', 'sum_L2_acc', 'sum_Linf_acc', 'sum_clean_acc', 'sum_common_acc']

# Create a list of unique backbones (regardless of loss)
unique_backbones = sorted(set(backbone for backbone, _, _, _ in df_per_row_sums.index))

# # Assign a unique color to each backbone
# color_palette = px.colors.qualitative.Plotly
# color_map = dict(zip(unique_backbones, color_palette * (len(unique_backbones) // len(color_palette) + 1)))

# Define line styles per loss function
line_styles = {
    'CLASSIC_AT': 'solid',
    'TRADES_v2': 'dot'
}

# Create the radar plot

from plotly.colors import qualitative
color_palette = qualitative.Bold

# Get unique backbones
unique_backbones = sorted(set(backbone for backbone, _, _, _ in df_per_row_sums.index))

# Assign colors per backbone
# color_map = dict(zip(unique_backbones, color_palette * (len(unique_backbones) // len(color_palette) + 1)))

fig = go.Figure()

for (backbone, loss, _, _), row in df_per_row_sums.iterrows():
    fig.add_trace(go.Scatterpolar(
        r=row.tolist() + [row.tolist()[0]],  # Close the loop
        theta=metrics + [metrics[0]],
        mode='lines',
        name=f"{backbone} ({loss})",
        line=dict(
            width=2,
            color=color_map[backbone],
            dash=line_styles.get(loss, 'solid')
        )
    ))

fig.update_layout(
    width=500, 
    height=500,
    paper_bgcolor='white',
    plot_bgcolor='white',
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 6],
            showline=True,
            linewidth=1,
            linecolor="lightgrey",
            gridcolor="lightgrey",
            gridwidth=0.5,
        ),
        angularaxis=dict(
            direction="clockwise",
            showline=True,
            linewidth=1,
            linecolor="lightgrey",
            gridcolor="lightgrey",
            gridwidth=0.5,
        ),
        bgcolor='white'
    ),
    legend=dict(
        x=0,
        y=1.0,
        xanchor='left',
        yanchor='top',
        bgcolor='rgba(255,255,255,0.9)',
        bordercolor='lightgrey',
        borderwidth=1,
        font=dict(size=11),
    ),
    showlegend=False,
    font=dict(color='black'),
    margin=dict(l=25, r=25, t=25, b=25)  # <<< reduced margins
)

fig.show()

fig.write_image("./radar_plot_{}_{}.png".format(size, pn2), scale=3  )                # upscale for higher DPI (1 = default)